In [51]:
# https://pyvis.readthedocs.io/en/latest/tutorial.html#getting-started
from pyvis.network import Network
import psycopg2
import pandas.io.sql as sqlio

In [52]:
conn = psycopg2.connect(
    dbname='dbname',
    user='dbuser',
    password='dbpwd',
    host='localhost',
    port='15432'
)
cursor = conn.cursor()

In [53]:
df = sqlio.read_sql_query("""
select h1.hashtag as hashtag_1, h2.hashtag as hashtag_2, hr.distance
from hashtags_relation hr
         join hashtag h1 on hr.hashtag_id_1 = h1.id
         join hashtag h2 on hr.hashtag_id_2 = h2.id
where hr.distance > 5
order by hr.distance DESC
limit 10000
""", conn)

In [54]:
display(df.head())

,hashtag_1,hashtag_2,distance
0,digitalart,digitalfashion,3093.0
1,digitalfashion,digitalart,3093.0
2,virtualfashion,digitalfashion,3090.0
3,digitalfashion,virtualfashion,3090.0
4,digitalfashion,clo3d,2979.0


In [55]:
got_net = Network(height='750px', width='100%', bgcolor='#222222', font_color='white')

In [56]:
# set the physics layout of the network
got_net.barnes_hut()

sources = df['hashtag_1']
targets = df['hashtag_2']
weights = df['distance']

edge_data = zip(sources, targets, weights)

In [57]:
for e in edge_data:
    src = e[0]
    dst = e[1]
    w = e[2]

    got_net.add_node(src, src, title=src)
    got_net.add_node(dst, dst, title=dst)
    got_net.add_edge(src, dst, value=w)

In [58]:
neighbor_map = got_net.get_adj_list()

# add neighbor data to node hover data
for node in got_net.nodes:
    node['title'] += ' Neighbors:<br>' + '<br>'.join(neighbor_map[node['id']])
    node['value'] = len(neighbor_map[node['id']])

In [59]:
got_net.show('insta_analytics.html')